# Install Semantic Link

In [ ]:
!pip install semantic-link -q

# Initialize Lakehouse Names

In [ ]:
raw_lakehouse = "lh_bronze"
clean_lakehouse = "lh_gold"

# Constants

In [ ]:
REPO_NAME = "Fabric-Cricsheet"
MAIN_NAME = f"/{REPO_NAME}-main"
FILES_PATH = '/Files'
GIT_PATH = '/git'
UTILITY_PATH = '/Utility Functions'
NOTEBOOK_PATH = "/Notebooks"
PACKAGE_FOLDER = 'packages'
FUNCTION_FOLDER = 'fabric_python_functions'
FABRIC_UTILS_FOLDER = "fabric_utils"
FABRIC_UTILS_FILE = f"{FABRIC_UTILS_FOLDER}.py"
CRICSHEET = {
        "Cricsheet Model": ["Cricsheet Analysis"]
    }
DATALOAD =  {
        "Data Load Model": ["Data Load Monitor"]
    }
GITHUB_REPO_ZIP = f"https://github.com/akhilannan/{REPO_NAME}/archive/refs/heads/main.zip"

# Get or create lakehouse details

In [ ]:
def get_or_create_lakehouse(lakehouse_name):
    try:
        lakehouse_details = mssparkutils.lakehouse.create(lakehouse_name)
    except:
        lakehouse_details = mssparkutils.lakehouse.get(lakehouse_name)
    workspace_id = lakehouse_details.workspaceId
    lakehouse_id = lakehouse_details.id
    abfss_lakehouse_path = f"abfss://{workspace_id}@onelake.dfs.fabric.microsoft.com/{lakehouse_id}"
    mount_point = f"/lakehouse/{lakehouse_name}"
    mssparkutils.fs.mount(abfss_lakehouse_path, mount_point)
    return mssparkutils.fs.getMountPath(mount_point)

# Create Mount Points

In [ ]:
# Semantic Models and Reports Structure
lh_semantic_reports = {
    clean_lakehouse: CRICSHEET,
    raw_lakehouse: DATALOAD
}
# Mount lakehouses and get paths
lakehouses = {}
# Mount lakehouses
for lh in lh_semantic_reports:
  lakehouses[lh] = get_or_create_lakehouse(lh)

# Set Variables for Semantic Model and Report

In [ ]:
semantic_model_report = []
for lakehouse, models in lh_semantic_reports.items():
    lh_files_path = lakehouses[raw_lakehouse] + FILES_PATH
    lh_git_path = lh_files_path + GIT_PATH 
    lh_repo_path = lh_git_path + MAIN_NAME
    for model, reports in models.items():
        bim_path = f"/Semantic Model/{model}.bim"
        for report in reports:
            report_path = f"/Power BI Report/{report}.Report/report.json"
            semantic_model_report.append({
                'lakehouse_name': lakehouse,
                'semantic_model': model,
                'semantic_model_bim': lh_repo_path + bim_path,
                'report_name': report,
                'report_json': lh_repo_path + report_path
            })

# Set Additional Variables

In [ ]:
import os
lh_git_notebook_path = lh_repo_path + NOTEBOOK_PATH
git_utils_path = lh_repo_path + UTILITY_PATH + '/' + FABRIC_UTILS_FILE
package_path = os.path.join(lh_files_path, PACKAGE_FOLDER)
function_folder_path = os.path.join(package_path, FUNCTION_FOLDER)
fabric_utils_path = os.path.join(function_folder_path, FABRIC_UTILS_FOLDER)

# Download Git Repo contents as Zip and Unzip it

In [ ]:
import requests
import zipfile
import io
import shutil

shutil.rmtree(lh_git_path, ignore_errors=True)
r = requests.get(GITHUB_REPO_ZIP)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(lh_git_path)

# Add and Import Fabric Utilities

In [ ]:
sc.addPyFile(git_utils_path)
import fabric_utils as U

# Create Wheel Package

In [ ]:
import subprocess
import sys

# Remove the existing package directory and create a new one
shutil.rmtree(package_path, ignore_errors=True)
os.makedirs(fabric_utils_path, exist_ok=True)

# Write the Python code to the functions.py file
python_file_path = os.path.join(fabric_utils_path, 'functions.py')
shutil.copyfile(git_utils_path, python_file_path)

# Extract the function names to a list
function_names =[]
with open(python_file_path, 'r') as file:
    for line in file:
        # Check if the line contains a function definition
        if line.startswith('def '):
            # Extract the function name
            function_name = line.split('def ')[1].split('(')[0]
            function_names.append(function_name)

# Write the __init__.py content
init_content = "\n".join(f"from .functions import {name}" for name in function_names)
init_file_path = os.path.join(fabric_utils_path, '__init__.py')
with open(init_file_path, 'w') as init_file:
    init_file.write(init_content)

# Write the setup.py content
setup_content = """
from setuptools import setup, find_packages

setup(
name='fabric_utils',
version='0.1',
packages=find_packages(),
install_requires=['semantic-link'],
)
"""
setup_file_path = os.path.join(function_folder_path, 'setup.py')
with open(setup_file_path, 'w') as setup_file:
    setup_file.write(setup_content)

# Build the wheel file
os.chdir(function_folder_path)
subprocess.check_call([sys.executable, 'setup.py', 'sdist', 'bdist_wheel'])

# Move the wheel file outside the 'fabric_utils' folder
wheel_files = [f for f in os.listdir('dist') if f.endswith('.whl')]
for file in wheel_files:
    shutil.move(f'dist/{file}', f'../{file}')

# Clean up the package directory
shutil.rmtree(function_folder_path)

# Print a success message
print("The wheel file has been successfully created and moved outside the 'fabric_utils' folder.")


# Copy Wheel file to Clean Lakehouse

In [ ]:
clean_package_path = package_path.replace(raw_lakehouse, clean_lakehouse)
if os.path.exists(clean_package_path):
        shutil.rmtree(clean_package_path)
shutil.copytree(package_path, clean_package_path)

# Deploy Notebook

In [ ]:
import json

# Define the code replacements using formatted strings
code_replacements = {
    'RAW_LAKEHOUSE = "lh_bronze"': f'RAW_LAKEHOUSE = "{raw_lakehouse}"',
    'CLEAN_LAKEHOUSE = "lh_gold"': f'CLEAN_LAKEHOUSE = "{clean_lakehouse}"'
}

# Define the mapping of notebook names to their respective default lakehouse
notebook_lakehouse = {
    'Cricsheet Orchestrator': raw_lakehouse,
    'Cricsheet Initialize': raw_lakehouse,
    'Cricsheet Ingest Data': raw_lakehouse,
    'Cricsheet Build Facts and Dimensions': clean_lakehouse,
    'Cricsheet Model Refresh': clean_lakehouse,
    'Cricsheet Optimize and Vacuum': raw_lakehouse
}

# Iterate over the notebook_lakehouse dictionary and process each notebook
for file_name, lakehouse_type in notebook_lakehouse.items():
    full_file_name = file_name + ".ipynb"
    file_path = os.path.join(lh_git_notebook_path , full_file_name)
    with open(file_path, 'r') as file:
        notebook_json = json.load(file)
    U.create_or_replace_notebook_from_ipynb(file_name, notebook_json, lakehouse_type, code_replacements)

# Deploy Semantic Model and Report

In [ ]:
# Define a function to process each semantic model report
def deploy_model_and_report(smr):
    # Load the BIM file JSON
    with open(smr['semantic_model_bim'], 'r') as file:
        bim_json = json.load(file)
    # Load the report JSON
    with open(smr['report_json'], 'r') as file:
        report_json = json.load(file)

    # Create or replace the semantic model from BIM
    U.execute_with_retries(U.create_or_replace_semantic_model_from_bim,
                           dataset_name=smr['semantic_model'],
                           bim_file_json=bim_json)
    # Update the model expression
    U.execute_with_retries(U.update_model_expression,
                           dataset_name=smr['semantic_model'],
                           lakehouse_name=smr['lakehouse_name'])
    # Create or replace the report from report JSON
    U.execute_with_retries(U.create_or_replace_report_from_reportjson,
                           report_name=smr['report_name'],
                           dataset_name=smr['semantic_model'],
                           report_json=report_json)

# Iterate over each semantic model report and process it
for smr in semantic_model_report:
    deploy_model_and_report(smr)

# Delete Git folder

In [ ]:
shutil.rmtree(lh_git_path, ignore_errors=True)